In [25]:
def preprocessor(df1):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    #df1=df1[df1['Status']=='Overdue']
    #print(df1['Status'])
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df2=df1[['Net.Due.Dt','Clearing','Status']]
    df2.dropna(inplace=True)
    df2['Range of Delay']=pd.to_datetime(df2['Clearing'],dayfirst=True)-pd.to_datetime(df2['Net.Due.Dt'],dayfirst=True)
    df2['Range of Delay']=df2['Range of Delay'].dt.days
    df2['Range of Delay']=pd.to_numeric(df2['Range of Delay'])
    df2 = df2.reset_index()
    df2.drop(columns = 'index',inplace=True)
    
    df2['Status'] = np.where(df2['Range of Delay'] > 7, 8,(np.where(df2['Range of Delay'] > 6, 7,(np.where(df2['Range of Delay'] > 5, 6,(np.where(df2['Range of Delay'] > 4, 5,(np.where(df2['Range of Delay'] > 3, 4,(np.where(df2['Range of Delay'] > 2, 3,(np.where(df2['Range of Delay'] > 1, 2,(np.where(df2['Range of Delay'] > 0, 1,0)))))))))))))))#, (np.where(df2['Range of Delay'] > 0, 1,0)))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 10, 2,(np.where(df2['Range of Delay'] > 0,1,0)))))))# (np.where(df2['Range of Delay'] > 30, 5, (np.where(df2['Range of Delay'] > 15, 4, (np.where(df2['Range of Delay'] > 7, 3, (np.where(df2['Range of Delay'] > 3, 2, (np.where(df2['Range of Delay'] > 0, 1 ,0)))))))))))))))
    #print(df2)
    print('Y s in the initial set')
    print(np.unique(df2['Status'],return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    df1['Status']=df2['Status']
    df1.dropna(inplace=True)
    y=df1['Status']
    
    
    busa = preprocessing.LabelEncoder()
    ccar= preprocessing.LabelEncoder()
    month=preprocessing.LabelEncoder()
    zone=preprocessing.LabelEncoder()
    bran=preprocessing.LabelEncoder()
    payt=preprocessing.LabelEncoder()
    
    df1.drop(columns='Status',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    arr=busa.fit(df1['BusA'])
    df1['BusA']=busa.transform(df1['BusA'])
    arr=ccar.fit(df1['CCAr'])
    df1['CCAr']=ccar.transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    arr=month.fit(df1['Month'])
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year','Clrng.doc.'],inplace=True)
    arr=zone.fit(df1['Zone'])
    df1['Zone']=zone.transform(df1['Zone'])    #.astype('category').cat.codes
    arr=bran.fit(df1['Bran'])
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    arr =payt.fit(df1['PayT'])
    df1['PayT'] = payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #print('Creating Red Alert Zone')
    #table = pd.crosstab(columns=y, index=df1['Account'])
    #table.columns= ["No delay"," 0 - 10", ">10"]
    #print(table)
    #print(table.sort_values(['>10'],ascending=False).head())
    #arr=table.sort_values(['>10'],ascending=False)
    #arr.drop(columns=["No delay"," 0 - 10"],inplace=True)
    #df_1=arr[arr['>10']>40]
    #df_2=[]
    
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    


    #print(df_1.index)
    
    
    ###################Oversampling##############################
    from imblearn.over_sampling import ADASYN 
    sm = ADASYN()
    df3, y = sm.fit_resample(df1, y)
    df1=pd.DataFrame(df3,columns=df1.columns)
    ############################################################
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    print('Y s in the set')
    print(np.unique(y,return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    return df1,y,busa,ccar,month,zone,bran,payt

In [26]:
def trainer(df1,y):
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import tree
    from sklearn import neighbors
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.model_selection import train_test_split
    
    #y=np.array(df1['Status'])
    
    

    X=df1
    #df1.head(5)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=0)
    # Random forest Model: 
    model= RandomForestClassifier(n_estimators = 100,random_state = 30,oob_score=True,n_jobs=-1,verbose=2,max_features=None)
    # Decion Tree Model: 
    decision_tree = tree.DecisionTreeClassifier(random_state=0, max_depth=100)
    # AdaBoost Classifier Model:
    clf = AdaBoostClassifier(base_estimator=model,n_estimators=100, random_state=0)
    # KNN :
    mo = neighbors.KNeighborsClassifier(n_neighbors =2)
    # Bagging Classifier Model:  
    bagging = BaggingClassifier(mo,max_samples=0.5, max_features=0.5)
    #Voting Classifier
    eclf = model#VotingClassifier(estimators=[ ('knn_bagging', bagging), ('adaboost', clf), ('RF',model)],voting='soft')
    eclf1 = eclf.fit(xtrain, ytrain)
    print('TRAINING DONE ......................')
    eclf1=eclf.predict(xtrain)
    eclf2=eclf.predict(xtest)
    print('TRAINING RESULTS-----------------------------------------')
    print("Accuracy: for Training :",metrics.accuracy_score(ytrain, eclf1)*100)
    print("Accuracy: for Testing :",metrics.accuracy_score(ytest, eclf2)*100)
    print()
    print('Confusion Matrix for Random Forest')
    print(metrics.confusion_matrix(eclf2, ytest))
    print()
    print('--------------------------------------------------------------------------------------------------------------------')
    return eclf

In [27]:
def predict(df1,clf,busa,ccar,month,zone,bran,payt):
    import numpy as np
    import pandas as pd
    import copy
    from sklearn import preprocessing
    
   
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    print(df1.shape)
    #df1['Status']=df1['Status'].astype('category').cat.codes)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    df=copy.deepcopy(df1)
    print(df.shape)
    
    df['Prediction']=np.zeros(len(df1))

    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=busa.transform(df1['BusA'])
    df1['CCAr']=ccar.transform(df1['CCAr'])
    #.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year'],inplace=True)
    df1['Zone']=zone.transform(df1['Zone'])#.astype('category').cat.codes
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    df1['PayT'] =payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    #df1.drop(columns=['Net.Due.Dt','Clearing'],inplace=True)
    #df1.drop(columns='Net.Due.Dt',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    
    
    print('Starting Prediction ....')
    prediction=clf.predict(df1)
    print('Generating Probabilities.....')
    predictprob=clf.predict_proba(df1)
    print('Finished prediction...')
    print()
    #df['Probability']=np.zeros(len(df))
    print(df1.head())
    

    arr=['No Delay','Next Day','Two Days','Three Days','Four Days' ,'Five Days' ,'Six Days' ,'Seven Days' ,'More than 7']
    #arr1=['Low delay','Low Medium Delay','High Delay']#,'High Delay']#,'30 to 180','>180']#,'8-15 days','16-30 days','31-60 days','61-90 days','90-180 days','>180 days']
    for i in range(len(arr)):
        df[arr[i]]=np.empty(len(df))
    print('Computing Prediction Table............')
    #for i in range(len(prediction)):
        #flag=0
        #if ( i % 500 == 0):
         #   print('----------------------------------------------------------------------------------------------------------- i =',i)
        #for k in range(len(redzone)):
           # if df['Account'][i] == redzone[k] :
             #   df['Prediction'][i]='>=10'
              #  flag=1
              #  print('----------------------------------------------------------------RED ALERT------------------------------------------ ')
                
        #if(flag == 0):
    
        #print(arr[prediction[i]])
        #arr2=""
        #print('Actual:',df['Arr..Clearing...Net.Due.Date.'][i])
        #print('--------------------------------------------------------')
    df['Prediction']=prediction
    #df['Prediction']=np.where(df['Prediction'] == 0, 'No Delay',(np.where(df['Prediction'] == 1, 'Low Delay',(np.where(df['Prediction'] == 2, 'High Delay','')))))
    for i in range(len(arr)):
        df[arr[i]]=np.around(predictprob[:,i],decimals=2)
        
    print(df.head(5))
            
    print()
    print(df.head())
    print('Writing to csv file')
    print('-----------------------------------------------------------------------------------------')
    df.to_csv('Predictedout5.csv',index=False)

In [28]:
# import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import copy
trainpath='My Data_Delay.csv'

print('Reading data...')
df1=pd.read_csv(trainpath)
print('Finished Reading...')
print()
print(df1.head(5))
print()

df3=copy.deepcopy(df1)
print('Preprocessing started .......')
df2,y,busa,ccar,month,zone,bran,payt=preprocessor(df1)
print(df2.head(5))
print('Preprocessing Done...........')
print('--------------------------------------------------------------------------------------------------------')
#print('RED ZONE VISUALISATION....')

#visualisation_red(df3)

Reading data...
Finished Reading...

     BusA CCAr     Reference   Account          Customer.Name   Zone  Bran  \
0  9600.0  BPR  1.115502e+09  928860.0  TATA STEEL PROCESSING  North  GBAD   
1  9600.0  BPR  1.115502e+09  928860.0  TATA STEEL PROCESSING  North  GBAD   
2  2500.0  BPR  2.410320e+09  928860.0  TATA STEEL PROCESSING  North  GBAD   
3  1000.0  BPR  2.412144e+09  928860.0  TATA STEEL PROCESSING  North  GBAD   
4  1000.0  BPR  2.412161e+09  928860.0  TATA STEEL PROCESSING  North  GBAD   

   Ty   Status  Local.Crcy.Amt  ...  Doc.Chq.dt Month Pstng.Date  Net.Due.Dt  \
0  RV  Overdue            0.04  ...  31.10.2018   Oct   31-10-18    10-11-18   
1  RV   Not OD            0.12  ...  05.11.2018   Nov   05-11-18    15-11-18   
2  RV  Overdue            0.16  ...  03.11.2018   Nov   03-11-18    13-11-18   
3  RV  Overdue            0.14  ...  31.10.2018   Oct   31-10-18    10-11-18   
4  RV   Not OD            0.14  ...  16.12.2018   Dec   16-12-18    26-12-18   

         G.L 

In [29]:
import time
t1=time.time()
print('Training Starting............')
clf=trainer(df2,y)
print('Training Done...................')
print('Time Taken',time.time()-t1)
print('-----------------------------------------------------------------------------------')

Training Starting............


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.2min


building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.0min finished


TRAINING DONE ......................


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   11.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   30.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    7.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   18.9s finished


TRAINING RESULTS-----------------------------------------
Accuracy: for Training : 97.74741200828157
Accuracy: for Testing : 91.10338164251208

Confusion Matrix for Random Forest
[[23985   278   233   101    61    56    28    12   132]
 [  606 23235  1351   293   114    49    34     4   146]
 [  377  1340 22107   987   293   129    55    24   268]
 [  247   353  1128 22323   732   232    99    88   602]
 [  175   389   347   939 23211   464   210    94  1257]
 [  168   156   229   315   505 24593   259   176   263]
 [   53    78   116   136   223   161 24854   206   105]
 [   48    21    67   186   169   144   220 25034   230]
 [  235   240   351   400   473   217    88   153 22815]]

--------------------------------------------------------------------------------------------------------------------
Training Done...................
Time Taken 749.594874382019
-----------------------------------------------------------------------------------


In [30]:
print('Feature Importances')
for feature in zip(df2.columns, clf.feature_importances_):
    print(feature)

Feature Importances
('BusA', 0.05245150971397869)
('CCAr', 0.0)
('Account', 0.15977221661408036)
('Zone', 0.021878850693367348)
('Bran', 0.03645488039369853)
('Local.Crcy.Amt', 0.18799999581118154)
('PayT', 0.061102949204081186)
('Month', 0.06826477657986513)
('Pstng.Date', 0.10514072765648562)
('Net.Due.Dt', 0.1350622969654681)
('Pending_Amy', 0.08411684202903764)
('pending_cnt', 0.08775495433875584)


In [41]:
testpath='Test.csv'
test=pd.read_csv(testpath)
predict(test,clf,busa,ccar,month,zone,bran,payt)
print() 
print('______________________________________  END  ____________________________________')
print()
print()

(3099, 21)
(3099, 21)
Starting Prediction ....


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


Generating Probabilities.....
Finished prediction...

   BusA  CCAr  Account  Zone  Bran  Local.Crcy.Amt  PayT  Month  Pstng.Date  \
0     0     0   929488     2    21            0.10    24      2    20181231   
1     0     0   928860     1     9            0.17    28      2    20181230   
2     0     0   928860     1     9            0.14    28      2    20181230   
3     0     0   928860     1     9            0.12    28      2    20181230   
4     2     0   928860     1     9            0.14    28      2    20181231   

   Net.Due.Dt  Pending_Amy  pending_cnt  
0    20190105         0.00            0  
1    20190109         0.00            0  
2    20190109         0.00            0  
3    20190109         0.00            0  
4    20190110         0.43            3  
Computing Prediction Table............
   BusA CCAr   Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  4132000216   929488      KAUSALYA AGENCIES  South  SBAD  RV   
1  1000  BPR  2412166200  

In [38]:
np.unique(test['PayT'],return_counts=True)

(array(['Z002', 'Z003', 'Z005', 'Z007', 'ZZ09', 'ZZ13', 'ZZ16', 'ZZ17',
        'ZZ18', 'ZZ19', 'ZZ20', 'ZZ21', 'ZZ28', 'ZZ29', 'ZZ30', 'ZZ32',
        'ZZ33', 'ZZ34', 'ZZ36', 'ZZ38', 'ZZ39', 'ZZ40', 'ZZ41', 'ZZ42',
        'ZZ81', 'ZZ82'], dtype=object),
 array([146,  58,   1,  97, 114,  49,  64,  13,  66, 473, 216,  36, 616,
         56, 143,  65,  11,  83,  21, 110,  15, 169, 380,  65,  17,  16],
       dtype=int64))

In [40]:
np.where(test['PayT']=='Z005')

(array([3028], dtype=int64),)